# Writing data to and reading data from a Database using Python

## Libraries and settings

In [1]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/Users/loic/Documents/data_analytics/w02/Aufgabe


## Create sqlite data base

In [2]:
# Create data base
conn = sqlite3.connect('supermarket_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

supermarket_database.db
apartment_database.db


## Create SQL-table in the database

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (Id VARCHAR(50),
                                                               Lat DECIMAL(20,15),
                                                               Lon DECIMAL(20,15),
                                                               Brand VARCHAR(50),
                                                               Shop VARCHAR(50),
                                                               City VARCHAR(50),
                                                               Street VARCHAR(50),
                                                               Housenumber VARCHAR(50),
                                                               Postcode VARCHAR(50),
                                                               Opening_hours VARCHAR(250))''')
# Confirm changes to the table
conn.commit()

## Read data from file to data frame

In [12]:
df = pd.read_csv('supermarkets_data_prepared.csv', 
                  sep=',', 
                  encoding='utf-8')#[['web-scraper-order', 'address_raw', 'rooms', 'area', 'price']]
print(df.shape)
df = df.drop(['type'], axis=1)
df.head(5)

(865, 11)


,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
1,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
2,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
3,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
4,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038,Mo-Sa 07:30-21:00


## Write data to the SQL-table in data base

In [13]:
df.to_sql(name = 'supermarkets_table',
          con = conn,
          index = False,
          if_exists = 'replace')

865

## Query the SQL-table

In [15]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM supermarkets_table''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Id','Lat','Lon','Brand','Shop', 'City', 'Street', 'Housenumber', 'Postcode', 'Opening_hours'])    
print(df.shape)
df

(865, 10)


,Id,Lat,Lon,Brand,Shop,City,Street,Housenumber,Postcode,Opening_hours
0,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
1,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
2,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
3,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
4,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038,Mo-Sa 07:30-21:00
...,...,...,...,...,...,...,...,...,...,...
860,10814018168,47.354212,8.436737,Migros,supermarket,Birmensdorf (ZH),Zürcherstrasse,15,8903,"Mo-Fr 08:00-20:00, Sa 08:00-18:00; PH off"
861,10814018169,47.353857,8.436716,Coop,supermarket,Birmensdorf (ZH),Zürcherstrasse,9,8903,Mo-Fr 08:00-20:00; Sa 8:00 - 18:00
862,10982669725,47.349782,8.258690,ALDI,supermarket,Villmergen,Grenzweg,3,5612,Mo-Sa 07:30-20:00
863,11025130806,47.059301,7.620697,Denner,supermarket,Burgdorf,Lyssachstrasse,27,3400,Mo-We 08:00-19:00; Th 08:00-20:00; Fr 08:00-21...


### Filter supermarkets in Winterthur

In [16]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM supermarkets_table
               WHERE city="Winterthur"''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Id','Lat','Lon','Brand','Shop', 'City', 'Street', 'Housenumber', 'Postcode', 'Opening_hours'])    
print(df.shape)
df

(16, 10)


,Id,Lat,Lon,Brand,Shop,City,Street,Housenumber,Postcode,Opening_hours
0,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
1,246533376,47.518953,8.720071,Migros,supermarket,Winterthur,Schaffhauserstrasse,152,8400,"Mo-Fr 08:30-20:00, Sa 08:00-18:00; PH off"
2,548919417,47.506863,8.712208,Migros,supermarket,Winterthur,Wülflingerstrasse,71,8400,"Mo-Fr 07:30-20:00, Sa 08:00-18:00; PH off"
3,1328945987,47.485289,8.762742,Migros,supermarket,Winterthur,Hinterdorfstrasse,40,8405,"Mo-We 08:00-19:00; Th,Fr 08:00-20:00; Sa 08:00..."
4,1519474175,47.485171,8.763196,Denner,supermarket,Winterthur,Hinterdorfstrasse,40,8405,Mo-We 08:30-19:00; Th-Fr 08:30-20:00; Sa 08:00...
5,3831772784,47.499163,8.721153,Migros,supermarket,Winterthur,Strickerstrasse,3,8400,"Mo-Fr 08:00-20:00, Sa 08:00-18:00; PH off"
6,4058248551,47.500117,8.731930,Migros,supermarket,Winterthur,Stadthausstrasse,31,8400,"Mo-Fr 07:00-20:00, Sa 08:00-18:00; PH off"
7,4095400190,47.497901,8.724756,ALDI,supermarket,Winterthur,Archplatz,2,8400,Mo-Sa 08:00-20:00
8,4379763119,47.496739,8.753231,Alnatura,supermarket,Winterthur,Industriestrasse,11,8400,"Mo-Fr 08:30-20:00, Sa 08:30-19:00; PH off"
9,4471606841,47.504229,8.771731,Migros,supermarket,Winterthur,Hintermühlenstrasse,4,8409,"Mo-Fr 08:00-20:00, Sa 08:00-18:00; PH off"


## Close db connection (if open)

In [17]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

False


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [18]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.0.0
Datetime: 2023-09-29 15:30:31
Python Version: 3.9.13
-----------------------------------
